# predicting OnMarket 

In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import train_test_split
from sklearn.compose import TransformedTargetRegressor
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
import datetime
from datetime import timedelta
import ast
import json    
import missingno as msno
import seaborn as sns
import arabic_reshaper
from bidi.algorithm import get_display
from sklearn.impute import SimpleImputer
# from sklearn_pandas import CategoricalImputer
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, precision_score, accuracy_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
# Helper packages
from plotnine import *

# Modeling packages
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.inspection import permutation_importance
from sklearn.inspection import partial_dependence
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns

from catboost import CatBoostRegressor, cv, Pool

from collections import defaultdict

from hyperopt import hp, fmin, tpe, STATUS_OK, STATUS_FAIL, Trials

import shap
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, BaggingRegressor
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.pipeline import Pipeline

from tqdm import tqdm


## i will use in all the models 5 cross vaildation and MRSE to measure the ERROR 

In [73]:
loss = 'neg_root_mean_squared_error'
k = 5 #k-fold

In [90]:
df = pd.read_csv('../data/ML_TEST2.csv')
df['user_id'] = df['user_id'].astype('object')

delete outliers as in the EDA

In [91]:
df.shape

(6519, 31)

In [92]:
df = df[df['isRent']==True]

In [94]:
# df =df[df['beds'] <=5]
# df = df.loc[(df['review'] > 3) | (df['review'] == 1)]
df = df[df['price'] <=100*1000]
# df = df[df['isRent']==True]
df = df[df['onMarket']<=65]

In [95]:
df.shape

(2575, 31)

In [96]:
df.drop(['createYEAR'],axis=1 ,inplace=True)
df.drop(['createDAY'],axis=1 ,inplace=True)
df.drop(['createMONTH'],axis=1 ,inplace=True)
df.drop(['lUpdateDAY'],axis=1 ,inplace=True)
df.drop(['lUpdateMONTH'],axis=1 ,inplace=True)
df.drop(['lUpdateYEAR'],axis=1 ,inplace=True)
df.drop(['id'],axis=1 ,inplace=True)

In [97]:

X = df.drop(['isRent' , 'DayOfRent' , 'timeBeforeRent'  , 'onMarket'  ] , axis=1)
y = df['onMarket']

In [98]:
X.columns

Index(['user_id', 'len_of_title', 'price', 'len_of_content', 'img_in_poss',
       'beds', 'livings', 'wc', 'area', 'street_width', 'age', 'ketchen', 'ac',
       'furnished', 'district', 'advertiser_type', 'review', 'profileImg',
       'iam_verified', 'zone'],
      dtype='object')

In [99]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
train_df = X_train.join(y_train)
test_df =X_test.join(y_test)

low varince

In [100]:
nzv = VarianceThreshold(threshold=0.1)

dealing with Skewness

In [101]:
yj = PowerTransformer(method="yeo-johnson")
# X_norm = ColumnTransformer([("norm", yj, selector(dtype_include="number"))])

Standardization

In [102]:
scaler = StandardScaler()

# standardize all numeric features
# std = ColumnTransformer([("norm", scaler, selector(dtype_include="number"))])

OneHotEncoder for advertiser_type

In [103]:
encoder = OneHotEncoder(handle_unknown="ignore")


rediction the dim

In [104]:
# pca = PCA(n_components=11)


In [105]:
preprocessor = ColumnTransformer(
  remainder="passthrough",
  transformers=[
  ("nzv_encode", nzv, selector(dtype_include="number")),
  ("norm", yj, selector(dtype_include="number")),
  ("std_encode", scaler, selector(dtype_include="number")),
  # ("pca_encode", pca, selector(dtype_include="number")),
  ("one-hot", encoder, selector(dtype_include="object")),
  ])

In [106]:
from sklearn.linear_model import RidgeCV
from sklearn.inspection import permutation_importance
from sklearn.inspection import partial_dependence

# RidgeCV

In [107]:
alphas = [1,10 , 20]
reg = RidgeCV(alphas= alphas , store_cv_values=True)

model_pipeline = Pipeline(steps=[
  ("preprocessor", preprocessor),
  ("reg", reg),
])
cb_reg_1 = model_pipeline.fit(X_train , y_train)
cv_MSE = np.mean(reg.cv_values_, axis=0)

In [108]:
alphas

[1, 10, 20]

In [109]:
cv_MSE

array([201.06323096, 211.19551535, 219.02940851])

In [110]:
reg.alpha_

1

In [111]:
results =cross_val_score(cb_reg_1, X_test, y_test.values.ravel(), cv=k, scoring=loss)
np.abs(np.mean(results))

15.440310275044942

# catBoost


In [144]:

cb_reg_1 = CatBoostRegressor( random_seed=13, verbose=200)

pipeline  = Pipeline(steps=[
  ("preprocessor", preprocessor),
  ("cb_reg_1", cb_reg_1),
])
REG1 = pipeline.fit(X_train , y_train)
# scores = cross_val_score(model_pipeline, X_train, y_train, cv = 5 )
# scores = GridSearchCV(model_pipeline, X_train, y_train , cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1)
results = cross_val_score(model_pipeline, X_test, y_test.values.ravel(), cv=k, scoring=loss)


Learning rate set to 0.045896
0:	learn: 16.0594654	total: 13.9ms	remaining: 13.8s
200:	learn: 13.1610672	total: 472ms	remaining: 1.88s
400:	learn: 11.9877392	total: 843ms	remaining: 1.26s
600:	learn: 11.0578084	total: 1.2s	remaining: 800ms
800:	learn: 10.2617796	total: 1.56s	remaining: 388ms
999:	learn: 9.5991463	total: 1.92s	remaining: 0us
Learning rate set to 0.03559
0:	learn: 15.8798058	total: 2.48ms	remaining: 2.48s
200:	learn: 12.7583025	total: 83.8ms	remaining: 333ms
400:	learn: 10.7241423	total: 162ms	remaining: 242ms
600:	learn: 9.1439012	total: 250ms	remaining: 166ms
800:	learn: 7.9355900	total: 330ms	remaining: 82ms
999:	learn: 6.9789078	total: 408ms	remaining: 0us
Learning rate set to 0.03559
0:	learn: 15.6321964	total: 755us	remaining: 755ms
200:	learn: 12.1696710	total: 99.2ms	remaining: 394ms
400:	learn: 10.3415303	total: 178ms	remaining: 266ms
600:	learn: 8.7829754	total: 266ms	remaining: 177ms
800:	learn: 7.5744356	total: 350ms	remaining: 86.9ms
999:	learn: 6.6391267	to

In [145]:

np.abs(np.mean(results))

16.42006388823208

-----------

In [119]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

# LGBMRegressor

In [120]:
from lightgbm import LGBMRegressor

lgbmRegressor = LGBMRegressor()
model_pipeline = Pipeline(steps=[
  ("preprocessor", preprocessor),
  ("lgbmRegressor", lgbmRegressor),
])
model_pipeline.fit(X_train, y_train)
results = cross_val_score(model_pipeline, X_test, y_test.values.ravel(), cv=k, scoring=loss)

np.abs(np.mean(results))

16.943182122238355


# XGBRegressor

In [146]:
from xgboost import XGBRegressor

In [147]:

kf = KFold(n_splits=5, shuffle=True, random_state=0)

# defining model
xgbreg = XGBRegressor()
model_pipeline = Pipeline(steps=[
  ("preprocessor", preprocessor),
  ("xgbreg", xgbreg),
])
# taking positive because cross val score returns -ve values
results = cross_val_score(model_pipeline, X_test, y_test.values.ravel(), cv=k, scoring=loss)

np.abs(np.mean(results))
# printing the output
# print(f"Scores - {scores}\nMean - {sum(scores)/len(scores)}")

15.973786138378694

In [52]:
# print(f"Scores - {results}\nMean - {sum(results)/len(results)}")

In [127]:
def Objective(trial):
    params = {
        "n_estimators": trial.suggest_categorical("n_estimators", [100,150, 200, 250]),
        "eta": trial.suggest_loguniform("eta",1e-2,0.1),
        "subsample": trial.suggest_discrete_uniform("subsample", 0.6,1,0.1),
        "min_child_weight": trial.suggest_int("min_child_weight",5,11,15),
        "colsample_bytree": trial.suggest_discrete_uniform("colsample_bytree", 0.6,1,0.1),
        "max_depth": trial.suggest_categorical("max_depth",[3,5,7,9,11,13]),
        "random_state": 2021
    }
    
    model = XGBRegressor(**params)
    
    model_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
     ("model", model),
    ])
    # taking positive because cross val score returns -ve values
    
    results = cross_val_score(model_pipeline, X_test, y_test.values.ravel(), cv=k, scoring=loss)

    
    
    return np.abs(np.mean(results))

In [128]:
import optuna

In [129]:
study = optuna.create_study(direction="minimize", study_name='Xgboost optimization')
study.optimize(Objective, n_trials=10)

[I 2022-08-24 02:29:25,913] A new study created in memory with name: Xgboost optimization
The distribution is specified by [5, 11] and step=15, but the range is not divisible by `step`. It will be replaced by [5, 5].
[I 2022-08-24 02:29:30,052] Trial 0 finished with value: 16.25195005795745 and parameters: {'n_estimators': 250, 'eta': 0.0773434357787042, 'subsample': 1.0, 'min_child_weight': 5, 'colsample_bytree': 0.6, 'max_depth': 13}. Best is trial 0 with value: 16.25195005795745.
The distribution is specified by [5, 11] and step=15, but the range is not divisible by `step`. It will be replaced by [5, 5].
[I 2022-08-24 02:29:32,038] Trial 1 finished with value: 16.317497172497568 and parameters: {'n_estimators': 150, 'eta': 0.07826974730208443, 'subsample': 0.8, 'min_child_weight': 5, 'colsample_bytree': 0.9, 'max_depth': 5}. Best is trial 0 with value: 16.25195005795745.
The distribution is specified by [5, 11] and step=15, but the range is not divisible by `step`. It will be replac

In [131]:
XGB_params = study.best_params

study.best_params

{'n_estimators': 200,
 'eta': 0.013720525474650126,
 'subsample': 1.0,
 'min_child_weight': 5,
 'colsample_bytree': 0.9,
 'max_depth': 5}

# LightGBM

In [134]:
import lightgbm as lgbm


In [135]:
lgbreg = lgbm.LGBMRegressor()
model_pipeline = Pipeline(steps=[
  ("preprocessor", preprocessor),
  ("lgbreg", lgbreg),
])
# taking positive because cross val score returns -ve values
results = cross_val_score(model_pipeline, X_test, y_test.values.ravel(), cv=k, scoring=loss)

np.abs(np.mean(results))

16.943182122238355

In [136]:
def Objective(trial):
    params = {
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }
    model = lgbm.LGBMRegressor(**params)
    model_pipeline = Pipeline(steps=[
  ("preprocessor", preprocessor),
  ("model", model),
])
    
    # taking positive because cross val score returns -ve values
    results = cross_val_score(model_pipeline, X_test, y_test.values.ravel(), cv=k, scoring=loss)


    
    return np.abs(np.mean(results))

In [137]:
study = optuna.create_study(direction="minimize", study_name='LGBM optimization')
study.optimize(Objective, n_trials=20)

[I 2022-08-24 02:32:46,741] A new study created in memory with name: LGBM optimization
[I 2022-08-24 02:32:46,957] Trial 0 finished with value: 15.835178761027686 and parameters: {'lambda_l1': 5.900653192269175, 'lambda_l2': 7.529964426217943e-06, 'num_leaves': 137, 'feature_fraction': 0.9920683569068947, 'bagging_fraction': 0.40191427483674125, 'bagging_freq': 7, 'min_child_samples': 46}. Best is trial 0 with value: 15.835178761027686.


[LightGBM] [Warning] lambda_l1 is set=5.900653192269175, reg_alpha=0.0 will be ignored. Current value: lambda_l1=5.900653192269175
[LightGBM] [Warning] feature_fraction is set=0.9920683569068947, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9920683569068947
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] lambda_l2 is set=7.529964426217943e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.529964426217943e-06
[LightGBM] [Warning] bagging_fraction is set=0.40191427483674125, subsample=1.0 will be ignored. Current value: bagging_fraction=0.40191427483674125
[LightGBM] [Warning] lambda_l1 is set=5.900653192269175, reg_alpha=0.0 will be ignored. Current value: lambda_l1=5.900653192269175
[LightGBM] [Warning] feature_fraction is set=0.9920683569068947, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9920683569068947
[LightGBM] [Warning] bagging_freq

[I 2022-08-24 02:32:47,174] Trial 1 finished with value: 16.058845736857144 and parameters: {'lambda_l1': 8.914312989446231e-06, 'lambda_l2': 1.2258965150224352e-07, 'num_leaves': 199, 'feature_fraction': 0.8571398052615085, 'bagging_fraction': 0.4327194292560218, 'bagging_freq': 4, 'min_child_samples': 54}. Best is trial 0 with value: 15.835178761027686.


[LightGBM] [Warning] lambda_l1 is set=8.914312989446231e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.914312989446231e-06
[LightGBM] [Warning] feature_fraction is set=0.8571398052615085, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8571398052615085
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] lambda_l2 is set=1.2258965150224352e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.2258965150224352e-07
[LightGBM] [Warning] bagging_fraction is set=0.4327194292560218, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4327194292560218
[LightGBM] [Warning] lambda_l1 is set=8.914312989446231e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.914312989446231e-06
[LightGBM] [Warning] feature_fraction is set=0.8571398052615085, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8571398052615085
[LightGBM] [Warni

[I 2022-08-24 02:32:47,429] Trial 2 finished with value: 16.95762089436989 and parameters: {'lambda_l1': 2.5930563611181126e-06, 'lambda_l2': 3.983262057609998e-06, 'num_leaves': 234, 'feature_fraction': 0.7287020084582607, 'bagging_fraction': 0.5549235633875929, 'bagging_freq': 7, 'min_child_samples': 15}. Best is trial 0 with value: 15.835178761027686.
[I 2022-08-24 02:32:47,628] Trial 3 finished with value: 15.96504428411921 and parameters: {'lambda_l1': 0.0015755628597179923, 'lambda_l2': 0.0010173527953246642, 'num_leaves': 21, 'feature_fraction': 0.997041262975757, 'bagging_fraction': 0.6730733047140018, 'bagging_freq': 5, 'min_child_samples': 62}. Best is trial 0 with value: 15.835178761027686.


...

[LightGBM] [Warning] lambda_l1 is set=0.00024583333013126444, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.00024583333013126444
[LightGBM] [Warning] feature_fraction is set=0.807252504696617, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.807252504696617
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] lambda_l2 is set=4.484914697364659e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.484914697364659e-07
[LightGBM] [Warning] bagging_fraction is set=0.8650294911226462, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8650294911226462
[LightGBM] [Warning] lambda_l1 is set=0.00024583333013126444, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.00024583333013126444
[LightGBM] [Warning] feature_fraction is set=0.807252504696617, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.807252504696617
[LightGBM] [Warning

[I 2022-08-24 02:32:50,921] Trial 18 finished with value: 15.91512097353812 and parameters: {'lambda_l1': 0.00010221431760009677, 'lambda_l2': 1.0111280673086898e-05, 'num_leaves': 225, 'feature_fraction': 0.658139537738784, 'bagging_fraction': 0.7318967051811494, 'bagging_freq': 5, 'min_child_samples': 85}. Best is trial 15 with value: 15.780724979410014.


[LightGBM] [Warning] lambda_l1 is set=0.00010221431760009677, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.00010221431760009677
[LightGBM] [Warning] feature_fraction is set=0.658139537738784, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.658139537738784
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] lambda_l2 is set=1.0111280673086898e-05, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0111280673086898e-05
[LightGBM] [Warning] bagging_fraction is set=0.7318967051811494, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7318967051811494
[LightGBM] [Warning] lambda_l1 is set=0.00010221431760009677, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.00010221431760009677
[LightGBM] [Warning] feature_fraction is set=0.658139537738784, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.658139537738784
[LightGBM] [Warni

[I 2022-08-24 02:32:51,160] Trial 19 finished with value: 15.796342571428251 and parameters: {'lambda_l1': 8.34406748970655e-07, 'lambda_l2': 1.8500314797125927e-05, 'num_leaves': 165, 'feature_fraction': 0.8834915205962548, 'bagging_fraction': 0.7809795819563845, 'bagging_freq': 2, 'min_child_samples': 70}. Best is trial 15 with value: 15.780724979410014.


[LightGBM] [Warning] lambda_l1 is set=8.34406748970655e-07, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.34406748970655e-07
[LightGBM] [Warning] feature_fraction is set=0.8834915205962548, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8834915205962548
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] lambda_l2 is set=1.8500314797125927e-05, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.8500314797125927e-05
[LightGBM] [Warning] bagging_fraction is set=0.7809795819563845, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7809795819563845
[LightGBM] [Warning] lambda_l1 is set=8.34406748970655e-07, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.34406748970655e-07
[LightGBM] [Warning] feature_fraction is set=0.8834915205962548, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8834915205962548
[LightGBM] [Warning] 

In [138]:
lgb_params = study.best_params
study.best_params

{'lambda_l1': 0.00022987187281534296,
 'lambda_l2': 2.6376895012299166e-07,
 'num_leaves': 178,
 'feature_fraction': 0.6201026375237395,
 'bagging_fraction': 0.8587670993238541,
 'bagging_freq': 3,
 'min_child_samples': 98}

-------

In [148]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor
final_estimator =  CatBoostRegressor( random_seed=13, verbose=200)

estimators = [('xgb', XGBRegressor( )),
              ('lgb', lgbm.LGBMRegressor()),
                  # ("reg", reg),
                ("lgbreg", lgbreg),
              ]
# model_pipeline = Pipeline(steps=[
# ("preprocessor", preprocessor),
# ("final_estimator", final_estimator),
# ])
reg = StackingRegressor(
        estimators=estimators,
         final_estimator=final_estimator)
model_pipeline = Pipeline(steps=[
("preprocessor", preprocessor),
("reg", reg),
])
cb_reg_1 = model_pipeline.fit(X_train , y_train)

Learning rate set to 0.045896
0:	learn: 15.9387945	total: 798us	remaining: 797ms
200:	learn: 13.3847009	total: 115ms	remaining: 456ms
400:	learn: 12.6592594	total: 231ms	remaining: 344ms
600:	learn: 12.1371334	total: 346ms	remaining: 229ms
800:	learn: 11.6944213	total: 461ms	remaining: 114ms
999:	learn: 11.2931755	total: 575ms	remaining: 0us


In [149]:
results = cross_val_score(cb_reg_1, X_test, y_test.values.ravel(), cv=k, scoring=loss)

np.abs(np.mean(results))

Learning rate set to 0.03559
0:	learn: 15.8634414	total: 1.01ms	remaining: 1.01s
200:	learn: 13.6540984	total: 75.2ms	remaining: 299ms
400:	learn: 12.3151200	total: 155ms	remaining: 232ms
600:	learn: 11.0926142	total: 244ms	remaining: 162ms
800:	learn: 10.1263698	total: 317ms	remaining: 78.9ms
999:	learn: 9.3129747	total: 388ms	remaining: 0us
Learning rate set to 0.03559
0:	learn: 15.6417712	total: 977us	remaining: 976ms
200:	learn: 12.8824709	total: 81.8ms	remaining: 325ms
400:	learn: 11.4059052	total: 152ms	remaining: 227ms
600:	learn: 10.0793169	total: 229ms	remaining: 152ms
800:	learn: 9.1084845	total: 306ms	remaining: 76ms
999:	learn: 8.2799781	total: 377ms	remaining: 0us
Learning rate set to 0.03559
0:	learn: 15.9424731	total: 456us	remaining: 456ms
200:	learn: 13.5872208	total: 72.8ms	remaining: 289ms
400:	learn: 12.2352024	total: 144ms	remaining: 215ms
600:	learn: 10.9923700	total: 239ms	remaining: 159ms
800:	learn: 10.0247802	total: 315ms	remaining: 78.3ms
999:	learn: 9.209545

16.55533020803083

In [150]:
import joblib
joblib.dump(cb_reg_1, "reg.pkl")

['reg.pkl']